In [1]:
import dt_apriltags as apriltag
import cv2
import annotation_utils as utils

import numpy as np
from scipy.spatial.transform import Rotation as R

import pyrealsense2 as rs
import urx
np.set_printoptions(suppress=True)

dist = np.array([[-0.04797802,  0.04744357,  0.00017416,  0.00067967, -0.00408397]])
# detector = apriltag.Detector(families="tagStandard52h13")
mtx = np.array([[633.09029639, 0., 629.06462963], [0., 638.7544391, 362.74013262], [0., 0., 1.]])

camera_params = [635.0, 635.0, 629.0646296262861, 362.7401326185789]
corners_3D = np.array(
[(0, 0,0),
 (24, 24,0),
 (24, -24,0),
 (-24, -24,0),
 (-24,24, 0)], dtype=float)

keypoints, anchors = utils.get_points_from_CAD()
anchors[:,2] +=1
keypoints = np.hstack((keypoints,np.ones((keypoints.shape[0],1))))

In [7]:
def solve_pnp(obj_points, imagePoints,mtx, dist,reprojectionError ):
    success, rvec, tvec,inliers  = cv2.solvePnPRansac(obj_points, np.array([imagePoints]), mtx, dist,reprojectionError = reprojectionError)
        
    if not success:
        print('not success in PnP')
        return
    return tvec, rvec,inliers

def get_socket_points(img, points_array, tvec, rvec, 
                      show=True):
    if isinstance(img, str):
        img = cv2.imread(img)
    img_points = None
    img_points, _ = cv2.projectPoints(points_array, rvec, tvec, mtx, dist)
    img_points = img_points.reshape(-1,2)

    if show:
        for p in img_points.reshape(-1,2):
            kp = int(p[0]), int(p[1])
            cv2.circle(img, kp, 2, (0, 0, 255), -1)
        cv2.imshow("Image", img)
        cv2.waitKey(0)
    cv2.destroyAllWindows()
    return img_points
def print_error_rotation(tf_error):
    vec = R.from_matrix(tf_error[:3,:3]).as_rotvec() 
    print("vector in marker", vec/np.linalg.norm(vec))
    print("angles, deg", np.linalg.norm(vec)*57)

def calculate_corrected_kts(tf_error,keypoints):
    keypoints3d_homog = np.hstack((
        keypoints,
        np.ones((keypoints.shape[0],1))))

    keypoints_corrected = (tf_error @ keypoints3d_homog.T).T
    return keypoints_corrected

def get_marker_pose(img_with_marker):
    res = utils.detect_apriltag(img_with_marker, camera_params)

    corners_2d = res[0].corners
    center_2d = res[0].center

    marker_pts_2d = np.vstack((center_2d, corners_2d))
    tvec_m,rvec_m,inliers_m = solve_pnp(corners_3D,marker_pts_2d,mtx, dist, 0.5)
    return tvec_m,rvec_m

In [8]:
def solve_correction(img_with_marker, anchor_gt_2D):
    def apply_correction():
        # rotation correction
        result = tf_s_in_c.copy()
        z_from = tf_s_in_c[:3,2]
        z_to = tf_m_in_cam[:3,2]
        angle = np.arccos(z_from@z_to)
        rot_ax = np.cross(z_from,z_to)

        rot_vec = rot_ax /np.linalg.norm(rot_ax) * angle
        rot_mtx = R.from_rotvec(rot_vec).as_matrix()
        # print(rot_mtx @ z_from -z_to )

        result[:3,:3]  = rot_mtx @ tf_s_in_c[:3,:3]
        # correction for OZ
        d_cam = - (tf_s_in_c[:3,3] - tf_m_in_cam[:3,3])
        z_proj = (d_cam) @ z_to
        corr_z = np.eye(4)
        corr_z[2,3] = z_proj

        return  result @ corr_z

    res = utils.detect_apriltag(img_with_marker, camera_params)

    corners_2d = res[0].corners
    center_2d = res[0].center

    marker_pts_2d = np.vstack((center_2d, corners_2d))
    tvec_m,rvec_m,inliers_m = solve_pnp(corners_3D,marker_pts_2d,mtx, dist, 0.5)
    # print("inliers_m\n",inliers_m.T)

    tf_m_in_cam = np.eye(4)
    tf_m_in_cam[:3,:3] = R.from_rotvec(rvec_m.flatten()).as_matrix()
    tf_m_in_cam[:3,3] = tvec_m.flatten()

    tvec_gt,rvec_gt,inliers_gt = solve_pnp(anchors,anchor_gt_2D,mtx, dist,1.0)
    # print("inliers_gt\n",inliers_gt.T)

    tf_s_in_c = np.eye(4)
    tf_s_in_c[:3,:3] = R.from_rotvec(rvec_gt.flatten()).as_matrix()
    tf_s_in_c[:3,3] = tvec_gt.flatten()
    tf_s_in_c_corrected = apply_correction()
    tf_err_in_m = np.linalg.inv(tf_m_in_cam) @ tf_s_in_c_corrected
    
    return tf_err_in_m

img_with_marker_name = 'marker_close_new_2.jpg'

img_with_marker = cv2.imread(img_with_marker_name)

anchor_gt_2D = np.array(
[[606, 323],
       [571, 277],
       [642, 277],
       [536, 323],
       [676, 322],
       [571, 383],
       [640, 383]], dtype=float)

img_without_marker_name = 'no_marker_close_new_2.jpg'

tf_err_in_m = solve_correction(img_with_marker, anchor_gt_2D)
corr_kpts = calculate_corrected_kts(tf_err_in_m,keypoints)


vector in marker [0. 0. 1.]
angles, deg 0.6554067255108319


In [18]:
img_with_marker = cv2.imread(img_with_marker_name)

tvec_m,rvec_m = get_marker_pose(img_with_marker)

img_without_marker = cv2.imread(img_without_marker_name)
get_socket_points(img_without_marker, corr_kpts[:,:3], tvec_m,rvec_m)
get_socket_points(img_without_marker, keypoints, tvec_m,rvec_m)

array([[573.83945364, 165.47394412],
       [627.4837454 , 164.90707743],
       [482.13230318, 205.42053634],
       [719.83236927, 202.78585672],
       [543.61911069, 212.49271149],
       [658.73539763, 211.19856003],
       [480.30397203, 246.796534  ],
       [722.63522939, 243.9694859 ],
       [434.79009499, 308.03149051],
       [769.37614423, 303.86724454],
       [516.904262  , 452.12237369],
       [691.39746624, 449.62782582],
       [514.85086018, 461.51077558],
       [693.69667411, 458.93197156]])

In [19]:
anchors_list = [
    [[654, 358], [630,325],[680,324],[606,359],[703,358],[631,399],[679,399]],
    [[652, 355],[630, 326],[673, 325],[609,356],[695, 356],[631, 393],[674, 393]],
    [[648, 355],[630, 329],[670,328],[611, 355],[688, 355],[630,389],[669, 389]],
    [[648,355],[630,331],[666, 331],[613, 355],[684,355],[630,386],[666, 386]],
    [[647, 355],[630,332],[663, 332],[614, 355],[679,355],[630,383],[663,383]]
]
pics_list = [44, 108, 172,236,300]

kpts_list = []
for i in range(5):
    img_with_marker_name = "data_for_correction/" + str(pics_list[i]) + '.jpg'
    img_without_marker_name = "data_for_correction/" + str(pics_list[i]) + ' (2).jpg'
    anchor_gt_2D = np.array(anchors_list[i])
    img_with_marker = cv2.imread(img_with_marker_name)
    img_without_marker = cv2.imread(img_without_marker_name)
    
    tf_err_in_m = solve_correction(img_with_marker, anchor_gt_2D)
    corr_kpts = calculate_corrected_kts(tf_err_in_m,keypoints)
    kpts_list.append(corr_kpts)

kpts_list = np.array(kpts_list)
corr_kpts_mean = np.mean(kpts_list, axis = 0)
print(corr_kpts_mean)

apriltag not found


TypeError: 'NoneType' object is not subscriptable

In [ ]:
anchor_kpts = np.zeros((6,2), dtype=float)
i = 0
anchor_kpts[2*i] = [6.143, 37.51]
anchor_kpts[2*i+1] = [-anchor_kpts[2*i][0], anchor_kpts[2*i][1]]
i +=1
anchor_kpts[2*i] = [33.75, 0]
anchor_kpts[2*i+1] = [-anchor_kpts[2*i][0], anchor_kpts[2*i][1]]
i+=1
anchor_kpts[2*i] = [19.9, -27.273]
anchor_kpts[2*i+1] = [-anchor_kpts[2*i][0], anchor_kpts[2*i][1]]

anchor_kpts = np.hstack((anchor_kpts, np.zeros((6,1))))
anchor_gt_kpts2D = np.array(
[(576, 266),
 (625, 267),
 (525, 349),
 (671, 353),
 (552, 409),
 (644, 409)], dtype=float)

In [ ]:
img_with_marker_name = 'marker.jpg'
img_without_marker_name = 'no_marker.jpg'
img = cv2.imread(img_without_marker_name)
utils.show_image_with_points(img,anchor_gt_kpts2D, "bb")
cv2.destroyAllWindows()

In [ ]:
obj_points = np.zeros((5,3), dtype=float)
obj_points[0][1] = 40.0
obj_points[1][0] = 40.0
obj_points[1][1] = 40.0
obj_points[2][0] = 40.0
obj_points[4][0] = 20.0
obj_points[4][1] = 20.0

print(obj_points.shape)

for r in results:
    imagePoints = r.corners.reshape(1,4,2)
    center = r.center.reshape(1,1,2)

    imgpoints = np.concatenate((imagePoints, center), axis=1)
    success, rvec, tvec = cv2.solvePnP(obj_points, imgpoints, mtx, dist)


In [ ]:
keypoints_new = (tf_s_in_c @ anchor_kpts.T).T
print(keypoints_new)
print(keypoints.shape)
keypoints = np.hstack((
    keypoints,
    np.zeros((keypoints.shape[0],1)),
    np.ones((keypoints.shape[0],1))))


keypoints_corrected = (tf_err_in_m @ keypoints.T).T

print(keypoints_corrected)

In [ ]:
anchor_gt_2D = np.array(
[(623, 336),
 (610, 318),
 (636, 318),
 (597, 336),
 (649, 336),
 (610, 358),
 (636, 358)], dtype=float)

In [ ]:
obj_points = np.zeros((5,3), dtype=float)
obj_points[0][1] = 40.0
obj_points[1][0] = 40.0
obj_points[1][1] = 40.0
obj_points[2][0] = 40.0
obj_points[4][0] = 20.0
obj_points[4][1] = 20.0

print(obj_points)

In [ ]:
def draw_nums(img, points, show=False):
    for i in range(len(points)):
        point = (int(points[i][0][0]), int(points[i][0][1]))
        cv2.putText(img, str(i), point, cv2.FONT_HERSHEY_SIMPLEX, 12, (255,0,255))
        cv2.circle(img, point, 2, (255,0,255), -1)
    
    if show:
        cv2.imshow('img', img)
        cv2.waitKey(10)

In [ ]:
def get_socket_points(img, points_array, lines_array, bbox_array, tvec, rvec, 
                      show=False, draw_l=True, draw_p=True, draw_b=True):
    if isinstance(img, str):
        img = cv2.imread(img)
#     img = increase_brightness(img, 50)
    img_points = None
    
    if draw_p:
        img_points, _ = cv2.projectPoints(points_array, rvec, tvec, mtx, dist)
        draw_points(img, img_points)
    if draw_l:
        lines_points, _ = cv2.projectPoints(lines_array, rvec, tvec, mtx, dist)
        draw_lines(img, lines_points)
    if draw_b:
        bbox_arr, _ = cv2.projectPoints(bbox_array, rvec, tvec, mtx, dist)
        draw_bbox(img, bbox_arr)
    
    if show:
        cv2.imshow("Image", img)
        cv2.waitKey(1)
    return img, img_points

In [ ]:
def detect_apriltag(img):
    if isinstance(img, str):
        print('reading img from file system')
        img = cv2.imread(img)
    if not type(img[0][0][0]) == np.uint8:
        img = img/img.max()
        img *= 255
        img = img.astype(np.uint8)
#         cv2.imshow('detect_apriltag', img)
        cv2.waitKey(1)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    results = detector.detect(gray,
                              estimate_tag_pose=True,
                              camera_params=camera_params, 
                              tag_size=0.04)
    if not results:
        print('apriltag not found')
        return
    return results

In [ ]:
def warp_img(image, res):
    corners = res[0].corners
    
    corners[:2] = np.array(sorted(corners[:2], key=lambda x: x[1]))
    corners[2:] = np.array(sorted(corners[2:], key=lambda x: x[1]))

    a = corners[:2][:, 0].mean()
    c = corners[2:][:, 0].mean()
    b = corners[::2][:, 1].mean()
    d = corners[1::2][:, 1].mean()

    desired_corners = np.array([[a, b], [a, d], [c, b], [c, d]], dtype=int)

    tform = get_projection(desired_corners, corners)
    image_warped = warp(image, tform)
    return image_warped, desired_corners

In [ ]:
def get_projection(c_to, c_from):
    tform = ProjectiveTransform()
    tform.estimate(c_to, c_from)
    return tform

In [ ]:
# pose_left = ur10.getl()

In [ ]:
ur10.movej((-4.048973385487692, -1.1045106093036097, 1.546417236328125,
            2.811586380004883, -0.7216728369342249, 3.1041338443756104))

In [ ]:
# pose_center = ur10.getl()

In [ ]:
# pose_right = ur10.getl()

In [ ]:
points_arr = np.array(
    [[-132. ,   44. ,  -46. ],
       [-132. ,   27. ,  -46. ],
       [-143. ,   46. ,  -43. ],
       [-143. ,   40. ,  -45. ],
       [-142.5,   31. ,  -46. ],
       [-142.5,   24. ,  -45. ],
       [-149. ,   60. ,  -45. ],
       [-149. ,   43. ,  -44. ],
       [-149. ,   27. ,  -45. ],
       [-149. ,   11. ,  -45. ],
       [-156. ,   47. ,  -46. ],
       [-156. ,   39. ,  -45. ],
       [-156. ,   31. ,  -46. ],
       [-156. ,   23. ,  -46. ],
       [-162. ,   35. ,  -45. ],
       [-171. ,   44. ,  -46. ],
       [-171. ,   27. ,  -45. ]], dtype=np.float32)
DEPTH = -40
bbox = np.array([
    [10, 128, DEPTH],
    [59, 128, DEPTH],
    [59, 170, DEPTH],
    [10, 170, DEPTH]
], dtype=float)
bbox[:,[0, 1]] = bbox[:,[1, 0]]
bbox[:, 0] = -bbox[:, 0]

In [ ]:
lines = np.array([
    [0,0,0],
    [100,0,0],
    [0,100,0],
    [0,0,100]
], dtype=float)

In [ ]:
def draw_bbox(img, bbox_arr, show=False):
    print(bbox_arr)
    color = (0,255,0)
    cv2.line(img, 
            [int(x) for x in bbox_arr[0][0]],
            [int(x) for x in bbox_arr[1][0]], color, 1)
    cv2.line(img, 
            [int(x) for x in bbox_arr[1][0]], 
            [int(x) for x in bbox_arr[2][0]], color, 1)
    cv2.line(img, 
            [int(x) for x in bbox_arr[2][0]], 
            [int(x) for x in bbox_arr[3][0]], color, 1)
    cv2.line(img, 
            [int(x) for x in bbox_arr[3][0]], 
            [int(x) for x in bbox_arr[0][0]], color, 1)
    
    if show:
        cv2.imshow('img', img)
        cv2.waitKey(10)

In [ ]:
def draw_lines(img, line_arr, show=False):
    colors = [(255,255,0), (0,255,0), (0,0,255)]
    for i, line in enumerate(line_arr[1:]):
        cv2.line(img, [int(x) for x in line_arr[0][0]], [int(x) for x in line[0]], colors[i-1], 2)
    if show:
        cv2.imshow('img', img)
        cv2.waitKey(10)

In [ ]:
cv2.destroyAllWindows()

In [ ]:
name = 2
count = 0
centers = []
# ress = []
while count < 10:
    img = get_photo_from_realsense()
    res = detect_apriltag(img)
    count += 1
    
    if not res: 
        continue

#     ress = res
#     break
#     centers.append(res[0].center)
#     print(res)
#     ress.append(res)
    
    draw_apriltag(img, res,show=True)
    tvec, rvec,objpts = solve_pnp(res)
    print(tvec)
#     processed, _ = get_socket_points(img, points_arr, lines, bbox, tvec, rvec, 
#                                    show=True, draw_l=True, draw_p=True, draw_b=False)
    
    
#     fname = f'/home/viacheslav/jupyter_notebooks/{name}_processed.jpg'
#     cv2.imwrite(fname, img)
#     name += 1
#     break


In [ ]:
[[-23.50502963]
 [  8.35980833]
 [110.64977061]]

In [ ]:
centers = np.array(centers)

print(np.var(centers,axis = 0))

In [ ]:
name = 1
ress1 = []
centers = []
corners = []
count = 0
while name< 100:
    img = get_photo_from_realsense()
    res = detect_apriltag(img)
#     print(res)
    if not res: 
        continue
    for r in res:
        centers.append(r.center)
        corners.append(r.corners)
    count+=1
#     ress.append(res)
#     cv2.imshow("Image", img)
#     cv2.waitKey(10)
    draw_apriltag(img, res)
#     tvec, rvec, objpts = solve_pnp(res)
#     processed, _ = get_socket_points(img, points_arr, lines, bbox, tvec, rvec, 
#                                    show=True, draw_l=True, draw_p=False, draw_b=False)
    
    
    fname = f'/home/viacheslav/jupyter_notebooks/new_data/{name}_processed.jpg'
#     print(fname)
    cv2.imwrite(fname, img)
    name += 1
#     break


In [ ]:
centers = np.array(centers)
print(np.mean(centers,axis = 0))
print(np.sqrt(np.var(centers,axis = 0)))

In [ ]:
corners = np.array(corners)
# i = 0
for i in range (4):
# print(np.mean(corners[:,i],axis = 0))
    print(np.sqrt(np.var(corners[:,i],axis = 0)))
# print(corners[0])